In [63]:
#Update your token
STUDENT_TOKEN = 'EUGENIO MARCHIORI'

## ignore this code, just used for submission
import requests
import pprint
import json
import random
import math
from copy import copy, deepcopy

def get_problem(problem_id):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/get-problem?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id))
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  return r.json()

def submit_answer(problem_id, answer):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/submit-answer?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id), json = answer)
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  result = r.json()['result']
  if result == 'PASSED':
    print('\033[92m' + result)
  else:
    print('\033[91m' + result)

In [1]:
def format_value(value):
  if value == '.':
    return '..'
  else:
    s = str(value)
    if len(s) == 1:
      s = ' ' + s
    return s

def format_h(const, max):
  s = ''
  for i in range(max - len(const)):
    s = s + '   '
  s = s + ' '.join(map(format_value, const))
  return s

def pretty_print(board, h_const, v_const):
  max_h = 1
  for c in h_const:
    max_h = max(max_h, len(c))
  max_v = 1
  for c in v_const:
    max_v = max(max_v, len(c))

  h_offset = ' '.join(map(lambda x: '  ', range(max_h)))
  for i in range(max_v):
    print('    ' + h_offset + ' '.join(map(lambda x: '  ' if i >= len(x) else format_value(x[i]), v_const)))
  for r in range(len(board)):
    row = board[r]
    print((h_offset if r ==0 else format_h(h_const[r-1], max_h)) + ' ' +  ' '.join(map(format_value, row)))

In [3]:
# Function to initialize the board state, which we might not have defined earlier in our conversation
new_problem = {
    'board': [['X', 'X', 'X'],
              ['X', '.', '.'],
              ['X', '.', '.']],
    'h_constraints': [[12], [6]],
    'id': 1,
    'v_constraints': [[10], [8]]
}

def initialize_board_state(problem):
    board = problem['board']
    h_constraints = problem['h_constraints']
    v_constraints = problem['v_constraints']

    # Initialize the possible values for each cell
    possible_values = {}
    for i, row in enumerate(board):
        for j, cell in enumerate(row):
            if cell == '.':  # If the cell is empty
                possible_values[(i, j)] = set(range(1, 10))

    # Apply constraints to the possible values
    for i, h_constraint in enumerate(h_constraints, 1):
        total = h_constraint[0]
        for j in range(1, len(board[0])):
            if board[i][j] == '.':
                possible_values[(i, j)] = {val for val in possible_values[(i, j)] if val <= total}

    for j, v_constraint in enumerate(v_constraints, 1):
        total = v_constraint[0]
        for i in range(1, len(board)):
            if board[i][j] == '.':
                possible_values[(i, j)] = {val for val in possible_values[(i, j)] if val <= total}

    stack = [(board, possible_values, 0)]
    return stack

# Initialize the board state for the new problem again
new_stack = initialize_board_state(new_problem)

# Now let's check the initial state of the new stack to ensure it's correct
initial_board_state, initial_possible_values, _ = new_stack[0]

# Print the initial possible values for each cell
initial_possible_values



      10  8
    X  X  X
12  X .. ..
 6  X .. ..


In [4]:
# Initialize the board state for the new problem again
new_stack = initialize_board_state(new_problem)

# Now let's check the initial state of the new stack to ensure it's correct
initial_board_state, initial_possible_values, _ = new_stack[0]

# Print the initial possible values for each cell
initial_possible_values


# Corrected function to check if a digit can be placed in a given cell
def can_place_digit(board, i, j, digit, h_constraints, v_constraints):
    # Check the horizontal constraint
    if i > 0:  # If not on the first row
        row_sum = digit + sum(value for x, value in enumerate(board[i]) if isinstance(value, int) and x != j)
        if row_sum > h_constraints[i-1][0]:
            return False

    # Check the vertical constraint
    if j > 0:  # If not on the first column
        column_sum = digit + sum(board[x][j] for x in range(len(board)) if isinstance(board[x][j], int) and x != i)
        if column_sum > v_constraints[j-1][0]:
            return False

    # Check the row for duplicate digits
    if digit in [board[i][x] for x in range(len(board[i])) if x != j]:
        return False

    # Check the column for duplicate digits
    if digit in [board[x][j] for x in range(len(board)) if x != i]:
        return False

    return True

# Function to place a digit on the board
def place_digit(board, i, j, digit):
    board[i][j] = digit

# Function to remove a digit from the board
def remove_digit(board, i, j):
    board[i][j] = '.'

# Function to check if the board is complete and valid
def is_complete_and_valid(board, h_constraints, v_constraints):
    for i in range(1, len(board)):  # Starting from 1 to skip the top row of 'X'
        row_sum = sum(value for value in board[i] if isinstance(value, int))
        if row_sum != h_constraints[i-1][0]:
            return False

    for j in range(1, len(board[0])):  # Starting from 1 to skip the leftmost column of 'X'
        column_sum = sum(board[x][j] for x in range(len(board)) if isinstance(board[x][j], int))
        if column_sum != v_constraints[j-1][0]:
            return False

    # If all constraints are satisfied, check if the board is filled
    for row in board:
        if '.' in row:
            return False

    return True

# Test the functions with a simple placement
test_board = [['X', 'X', 'X', 'X', 'X'],
              ['X', 'X', 'X', '.', '.'],
              ['X', '.', '.', '.', '.'],
              ['X', '.', '.', '.', '.'],
              ['X', '.', '.', 'X', 'X']]
test_h_constraints = [[5], [13], [14], [11]]
test_v_constraints = [[23], [7], [6], [7]]

# Test placing a digit
place_digit(test_board, 2, 1, 9)
assert can_place_digit(test_board, 2, 1, 9, test_h_constraints, test_v_constraints) == True
remove_digit(test_board, 2, 1)
assert test_board[2][1] == '.'
assert is_complete_and_valid(test_board, test_h_constraints, test_v_constraints) == False

"Corrected functions are successfully implemented."


'Corrected functions are successfully implemented.'

In [5]:
def calculate_mrv(possible_values):
    """
    Calculate the Minimum Remaining Values for each cell.
    :param possible_values: A dictionary with keys as cell coordinates and values as a set of possible digits.
    :return: A list of cells sorted by their MRV.
    """
    mrv = sorted(possible_values.items(), key=lambda item: len(item[1]))
    return mrv

# Initialize the board state for the new problem again
new_stack = initialize_board_state(new_problem)

# Now let's check the initial state of the new stack to ensure it's correct
initial_board_state, initial_possible_values, _ = new_stack[0]

# Print the initial possible values for each cell
initial_possible_values


{(1, 1): {1, 2, 3, 4, 5, 6, 7, 8, 9},
 (1, 2): {1, 2, 3, 4, 5, 6, 7, 8},
 (2, 1): {1, 2, 3, 4, 5, 6},
 (2, 2): {1, 2, 3, 4, 5, 6}}

In [6]:
def solve_kakuro_debug(stack, h_constraints, v_constraints):
    backtracking_count = 0

    while stack:
        # Pop the current state from the stack
        current_board, current_possible_values, backtracking_count = stack.pop()
        
        # Calculate MRV
        mrv_cells = calculate_mrv(current_possible_values)

        for cell, values in mrv_cells:
            i, j = cell
            print(f"Trying cell ({i}, {j}) with possible values {values}")
            for digit in values:
                if can_place_digit(current_board, i, j, digit, h_constraints, v_constraints):
                    print(f"Placing {digit} in cell ({i}, {j})")
                    new_board = [row[:] for row in current_board]  # Deep copy to avoid mutation
                    new_possible_values = {k: v.copy() for k, v in current_possible_values.items()}  # Copy possible values
                    place_digit(new_board, i, j, digit)
                    # Remove the placed digit from possible values for other cells
                    new_possible_values.pop((i, j))
                    for k in new_possible_values:
                        new_possible_values[k] -= {digit}
                    stack.append((new_board, new_possible_values, backtracking_count))
                    break  # Only place one digit before checking the board again
                else:
                    print(f"Cannot place {digit} in cell ({i}, {j})")
            else:
                # If no digit can be placed, need to backtrack
                backtracking_count += 1
                print(f"Backtracking at cell ({i}, {j}), count: {backtracking_count}")

            # Check if we've found a solution
            if is_complete_and_valid(current_board, h_constraints, v_constraints):
                return current_board, backtracking_count

    # If the stack is empty and no solution was found, return None
    return None, backtracking_count



In [8]:
# New problem setup
new_problem = {
    'board': [['X', 'X', 'X'],
              ['X', '.', '.'],
              ['X', '.', '.']],
    'h_constraints': [[12], [6]],
    'id': 1,
    'v_constraints': [[10], [8]]
}

# Initialize the board state for the new problem
new_stack = initialize_board_state(new_problem)

# Print the initial board state using the pretty_print function
pretty_print(new_problem['board'], new_problem['h_constraints'], new_problem['v_constraints'])

# Now let's run the solver with the new initial stack
new_solution_board, new_total_backtracking = solve_kakuro_debug(new_stack, new_problem['h_constraints'], new_problem['v_constraints'])

# Print the solution or indicate if there is no solution
if new_solution_board:
    print("Solution")
    pretty_print(new_solution_board, new_problem['h_constraints'], new_problem['v_constraints'])
else:
    print("No solution was found.")


      10  8
    X  X  X
12  X .. ..
 6  X .. ..
Trying cell (2, 1) with possible values {1, 2, 3, 4, 5, 6}
Placing 1 in cell (2, 1)
Trying cell (2, 2) with possible values {1, 2, 3, 4, 5, 6}
Placing 1 in cell (2, 2)
Trying cell (1, 2) with possible values {1, 2, 3, 4, 5, 6, 7, 8}
Placing 1 in cell (1, 2)
Trying cell (1, 1) with possible values {1, 2, 3, 4, 5, 6, 7, 8, 9}
Placing 1 in cell (1, 1)
Trying cell (2, 1) with possible values {2, 3, 4, 5, 6}
Placing 2 in cell (2, 1)
Trying cell (2, 2) with possible values {2, 3, 4, 5, 6}
Placing 2 in cell (2, 2)
Trying cell (1, 2) with possible values {2, 3, 4, 5, 6, 7, 8}
Placing 2 in cell (1, 2)
Trying cell (2, 1) with possible values {3, 4, 5, 6}
Placing 3 in cell (2, 1)
Trying cell (2, 2) with possible values {3, 4, 5, 6}
Placing 3 in cell (2, 2)
Trying cell (2, 1) with possible values {4, 5, 6}
Cannot place 4 in cell (2, 1)
Cannot place 5 in cell (2, 1)
Cannot place 6 in cell (2, 1)
Backtracking at cell (2, 1), count: 1
Trying cell (2, 2)